**Extinction Analysis Notebook**

This notebook performs an analysis of extinction dynamics in a predator-prey system. The primary goal is to determine the proportion of initial conditions that lead to predator extinction under specific parameter settings. The analysis involves:

1. **Random Initialization**:
   - Generating random initial conditions for prey densities (ρ₁ and ρ₂) and predator group densities (γₓ).

2. **Simulation of Dynamics**:
   - Using a bounded initial value problem (IVP) solver to simulate the system dynamics over a fixed number of iterations.

3. **Extinction Criteria**:
   - Defining extinction based on the population falling below a predefined threshold, and using this to classify whether the iterations led to one of the 6 types of extinction equilibria.

   
5. **Parallel Execution**:
   - Leveraging multiprocessing to run simulations in parallel, significantly reducing computation time for large numbers of initial conditions.

6. **Output and Results**:
   - Calculating the proportion of initial conditions that result in predator extinction and documenting the results.

The notebook integrates functions from the `group_w_pop_funs` and `sim_graph_funs` modules to handle system dynamics and random initialization respectively. It is structured for reproducibility and extensibility for further exploration of extinction scenarios under varying parameters.



**Types of Extinction Equilibria**
1.  Only predators extinct.
1. Predators extinct and both prey extinct
1. Predators extinct and big prey, but not small prey, extinct
1. Predators extinct and small prey, but not big prey, extinct
1. Only big prey is extinct
1. Only small prey is extinct

In [1]:
import numpy as np
from multiprocessing import Pool, cpu_count
final_fig_path = "../CH_Manuscript/Figures/"
import sys
sys.path.insert(1, 'Functions')
from extinction_utils import extinction_analysis_multiprocessing
# from group_w_pop_funs import bounded_ivp  # Ensure this function works with multiprocessing
from sim_graph_funs import get_initial_points, update_params
from equilibria_funs import initiate_g_first_x

This is what chat gpt wrote:

In [2]:
H=1
x_max = 5
params_base = dict(η1 = 0.2, η2 = 0.5, A = 0.5, β1 = 8, β2 = 1, H1=H, H2=H, 
                  α1_of_1=0.05, α2_of_1=0.95, 
                  s1=2, s2=2, α2_fun_type = 'constant',
                  x_max = x_max, d = 10,
                 Tx = .01, pop_process = True)

params = update_params("scale", 4, params_base)
initialstate = [.3,.7,*initiate_g_first_x(3,params["x_max"])]

args = (initialstate, 1000, params, 1e-6)

In [6]:
if __name__ == "__main__":
    H=1
    x_max = 5
    params_base = dict(η1 = 0.2, η2 = 0.5, A = 0.5, β1 = 8, β2 = 1, H1=H, H2=H, 
                      α1_of_1=0.05, α2_of_1=0.95, 
                      s1=2, s2=2, α2_fun_type = 'constant',
                      x_max = x_max, d = 10,
                     Tx = .01, pop_process = True)
    
    params = update_params("scale", 4, params_base)
    initialstate = [.3,.7,*initiate_g_first_x(3,params["x_max"])]
    num_points = 1000
    t_f = 1000
    extinction_threshold = 1e-6
    results = extinction_analysis_multiprocessing(
        num_points, t_f, params, extinction_threshold)
    print(results)


{'predator_extinct_both_prey_extinct': 0.0, 'predator_extinct_big_prey_extinct': 0.0, 'predator_extinct_small_prey_extinct': 0.0, 'predator_extinct': 1.0, 'big_prey_extinct': 0.0, 'small_prey_extinct': 0.0, 'no_extinction': 0.0}


# ChatGPT suggestion for doing this over a whole parameter grid

# Parameter Grid Analysis for Extinction Dynamics

This section performs extinction analysis across a grid of parameter combinations. The goal is to evaluate how different parameter settings influence the extinction dynamics.

### Steps:

1. **Define Parameter Ranges**:
   - Specify the ranges for parameters such as \( \eta_1, \eta_2, A, d, T_x, \text{scale} \).
   - Ensure that constraints like \( \eta_1 < \eta_2 \) and relationships like \( \text{scale} = \frac{H_1}{H_2} = \frac{\beta_1}{\beta_2} \) are maintained.

2. **Generate Parameter Grid**:
   - Use `itertools.product` to generate all possible combinations of parameters.
   - Filter combinations to enforce constraints, e.g., \( \eta_1 < \eta_2 \).

3. **Define Helper Function for Analysis**:
   - The `analyze_params` function takes a single parameter combination as input.
   - Computes extinction proportions using the `extinction_analysis_multiprocessing` function.
   - Returns the parameter combination and extinction results.

4. **Parallelize Computation**:
   - Use Python’s `multiprocessing.Pool` to distribute the analysis across multiple CPU cores.
   - This significantly reduces computation time for large grids.

5. **Save Results**:
   - Store the results as a JSON file for further analysis or visualization.

### Code Example:

```python
import numpy as np
from itertools import product
from multiprocessing import Pool, cpu_count
from group_w_pop_funs import bounded_ivp
from sim_graph_funs import get_initial_points

# Define the parameter grid
eta1_values = np.linspace(0.01, 0.99, 5)  # Avoid 0 and 1
eta2_values = np.linspace(0.01, 0.99, 5)
A_values = np.linspace(0.1, 1.5, 5)
d_values = np.arange(1, 6)  # Positive integers
Tx_values = np.linspace(0.1, 5, 5)  # Positive values
scale_values = np.linspace(0.5, 2.0, 5)  # Scale (H1/H2 or beta1/beta2)

# Fixed parameters
alpha1_of_1 = 0.05
alpha2_of_1 = 0.95
s1, s2 = 2, 2

# Helper function to run analysis for one parameter set
def analyze_params(params):
    eta1, eta2, A, d, Tx, scale = params
    beta1 = 0.5  # Arbitrary base value
    beta2 = beta1 / scale  # Maintain β1/β2 = H1/H2
    H1 = 1.0  # Arbitrary base value for handling scale
    H2 = H1 / scale

    extinction_params = {
        "eta1": eta1,
        "eta2": eta2,
        "A": A,
        "beta1": beta1,
        "beta2": beta2,
        "H1": H1,
        "H2": H2,
        "alpha1_of_1": alpha1_of_1,
        "alpha2_of_1": alpha2_of_1,
        "s1": s1,
        "s2": s2,
        "d": d,
        "Tx": Tx,
        "x_max": 10  # Example max group size
    }

    num_points = 100  # Number of initial conditions
    t_f = 1000  # Time horizon
    extinction_threshold = 1e-6  # Threshold for extinction

    # Perform extinction analysis
    proportions = extinction_analysis_multiprocessing(
        num_points=num_points,
        t_f=t_f,
        params=extinction_params,
        extinction_threshold=extinction_threshold
    )
    return (params, proportions)

# Create the grid of parameters with η1 < η2
parameter_grid = [
    (eta1, eta2, A, d, Tx, scale)
    for eta1, eta2, A, d, Tx, scale in product(eta1_values, eta2_values, A_values, d_values, Tx_values, scale_values)
    if eta1 < eta2  # Enforce η1 < η2
]

# Use multiprocessing to analyze the grid
if __name__ == "__main__":
    n_jobs = cpu_count()  # Adjust the number of parallel processes
    with Pool(processes=n_jobs) as pool:
        results = pool.map(analyze_params, parameter_grid)

    # Save results to file
    import json
    with open("extinction_analysis_results.json", "w") as f:
        json.dump(results, f)


# Importing and Analyzing Extinction Analysis Results from JSON

This section describes how to import and work with data from the JSON file generated by the extinction analysis.

### Steps:

1. **Load the JSON File**:
   - Use Python's built-in `json` module to load the data from the file.
   - Convert the JSON format into a Python list for easy manipulation.

2. **Inspect the Data**:
   - Each entry in the list contains:
     - A tuple of parameters (`params`), representing the parameter combination used in the analysis.
     - A dictionary of extinction proportions (`proportions`), showing the fraction of simulations that ended in each type of extinction.

3. **Iterate Through the Results**:
   - Loop through the list to extract and analyze the parameter combinations and extinction proportions.
   - Example analyses include filtering results by specific parameters or calculating summary statistics.

### Code Example:

```python
import json

# Load the JSON file
file_path = "extinction_analysis_results.json"  # Replace with your actual file path
with open(file_path, "r") as f:
    data = json.load(f)

# Example: Inspect the first entry
print("First entry in the results:")
print(data[0])

# Example: Iterate through the results
for params, proportions in data:
    print(f"Parameters: {params}")
    print(f"Extinction Proportions: {proportions}")
